In [1]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

In [2]:
client_id = '529eee33741c45de9d12777f9b4d8f2d'
client_secret = '0c9d6e87495e45d69e3afd78356d0c23'

creds = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=creds)

In [3]:
#refresh token as token expires after 1 hour
#token = creds.get_access_token()
#sp = spotipy.Spotify(auth=token)

In [4]:
#function to grab playlist from user
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids = getTrackIDs('United States Top 50', '37i9dQZEVXbLRQDuF5jeBp')

In [5]:
#grab data from track
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

    #main track items
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    #track features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    instrumentalness = features[0]['instrumentalness']
    loudness = features[0]['loudness']
    tempo = features[0]['tempo']
    speechiness = features[0]['speechiness']
    valence = features[0]['valence']
    time_signature = features[0]['time_signature']
    energy = features[0]['energy']
    liveness = features[0]['liveness']


    track = [name, album, artist, release_date, length, popularity, acousticness, danceability, instrumentalness, loudness, tempo, speechiness, valence, time_signature, energy, liveness]
    return track

In [6]:
#grab tracks
tracks = []

for i in range(len(ids)):
    time.sleep(.5)
    track = getTrackFeatures(ids[i])
    tracks.append(track)

In [7]:
current50 = pd.DataFrame(tracks,
                 columns = ['name',
                            'album',
                            'artist',
                            'release_date',
                            'length',
                            'popularity',
                            'acousticness',
                            'danceability',
                            'instrumentalness',
                            'loudness',
                            'tempo',
                            'speechiness',
                            'valence',
                            'time_signature',
                           'energy',
                           'liveness'])

## What Makes A Song Popular?

In [8]:
stored_songs = pd.read_csv(r'D:\Data Science\Spotify API\data\stored_songs.csv')
stored_songs.drop('Unnamed: 0', axis = 1, inplace=True)

In [9]:
#Need to add songs new to current50 list to stored_songs
stored_songs = pd.concat([stored_songs, current50], ignore_index=True, sort=True)

In [10]:
stored_songs.drop_duplicates(inplace=True)
stored_songs['genre'] = None

In [11]:
#format stored songs to merge with past popular songs
order =                     ['name',
                            'artist',
                             'album',
                             'genre',
                            'release_date',
                             'length',
                            'acousticness',
                            'danceability',
                            'instrumentalness',
                            'loudness',
                             'liveness',
                            'tempo',
                             'energy',
                            'speechiness',
                            'valence',
                            'time_signature',
                            'popularity']
stored_songs = pd.DataFrame(stored_songs, columns = order)

In [12]:
songs10_19 = pd.read_csv(r"D:\Data Science\Spotify API\data\2010-2019_pop_songs.csv", encoding='ISO-8859-1')
songs10_19.drop('Unnamed: 0',axis = 1, inplace=True)
songs10_19['album'] = None
songs10_19['instrumentalness'] = None
songs10_19['time_signature'] = None

In [13]:
columns = ['name',
            'artist',
           'genre',
            'release_date',
            'tempo',
            'energy',
            'danceability',
            'loudness',
            'liveness',
            'valence',
            'length',
            'acoustiness',
            'speechiness',
            'popularity',
            'album',
            'instrumentalness',
            'time_signature']
songs10_19.columns = [i for i in columns]

In [14]:
songs10_19 = pd.DataFrame(songs10_19, columns = order)

## Extracting Missing Data From Spotify API 

In [15]:
#get missing data
def artist_info(lookup) :

    try :
        artist = sp.search(lookup)
        artist_uri = artist['tracks']['items'][0]['album']['artists'][0]['uri']
        track_uri = artist['tracks']['items'][0]['uri']

        release_date = artist['tracks']['items'][0]['album']['release_date']
        album = artist['tracks']['items'][0]['album']

        artist = sp.artist(artist_uri)
        popularity = artist['popularity']

        audio_features = sp.audio_features(track_uri)[0]

        acousticness = audio_features['acousticness']
        instrumentalness = audio_features['instrumentalness']
        time_signature = audio_features['time_signature']

        return popularity, acousticness, instrumentalness, time_signature, album

    except :
        return [None]*5

In [16]:
songs10_19['lookup'] = songs10_19['name'] + '' + songs10_19['artist']

In [17]:
songs10_19['popularity_2'], songs10_19['acousticness'], songs10_19['instrumentalness'], songs10_19['time_signature'], songs10_19['album'] = zip(*songs10_19['lookup'].map(artist_info))

In [18]:
top_songs = pd.concat([stored_songs, songs10_19], ignore_index = True, sort = True)

In [19]:
songs10_19.drop('lookup', axis = 1, inplace=True)

In [30]:
#read data from github
old_popu_songs = pd.read_csv(r'https://raw.githubusercontent.com/ZacharyJWyman/Spotify-API/master/data/songs10_19songs.csv?token=APIXTT7OQYTJIHI3JSIRO427H3YA2')
new_popu_songs = pd.read_csv(r'https://raw.githubusercontent.com/ZacharyJWyman/Spotify-API/master/data/stored_songs.csv?token=APIXTT6HY3BVIA5DVBXWR6C7H3X7I')